In [ ]:
## SCHEMA MEtHODO : SCRAPPING, TEXT MINING, TF-IDF, LDA, BERTopic, COMPARAISON LDA/BERTopic, LINK ANALYSIS

In [ ]:

##fltrer pour garder 'les vrais documents'## --> au cas ou on a trop il faut encore filtrer sinon on ne sait pas utiliser ( avoir avec tt les liens)
df_final["word_count"] = df_final["cleaned"].apply(lambda x: len(str(x).split()))
df_articles = df_final[df_final["word_count"] >= 400].copy()  # seuil à ajuster au cas ou pour enlever le bruit
print("Docs gardés:", len(df_articles))


In [ ]:
## Vectoriser (Bag-of-Words) pour LDA
'''from sklearn.feature_extraction.text import CountVectorizer''' #--> déja ds le code d eury mais utilisé ici

vectorizer = CountVectorizer(
    stop_words="english",
    min_df=5,      # mot présent dans au moins 5 docs ####--> a modifier en focntion du nombre d'article
    max_df=0.8     # ignore les mots trop fréquents ####--> a modifier en focntion du nombre d'article
)

X = vectorizer.fit_transform(df_articles["cleaned"])


In [ ]:
## LDA

from sklearn.decomposition import LatentDirichletAllocation 

n_topics = 6  # bon point de départ pour lifestyle # a modifier au cas ou en fonction de tout nos liens --> si 2 mot topique trop similaire nombre trop grand

lda = LatentDirichletAllocation(
    n_components=n_topics,
    random_state=42
)
lda.fit(X)


In [ ]:
## afficher les topics (top mots)

'''import numpy as np ''' #--> déja ds le code d eury mais utilisé ici

feature_names = vectorizer.get_feature_names_out() # Récupérer la liste des mots du vocabulaire appris par le vectorizer

def show_topics(model, feature_names, n_top_words=10): 
    topics = []
    for topic_idx, topic in enumerate(model.components_):# Parcourir chaque topic appris par le modèle
                                        # model.components_ = matrice (topics x mots)
        top_idx = topic.argsort()[:-n_top_words-1:-1] # Trier les mots du topic par importance décroissante 
                        # argsort() donne les indices triés → on prend les plus grands poids
        top_words = [feature_names[i] for i in top_idx] # Convertir les indices en mots lisibles
        topics.append((topic_idx, top_words))  # Sauvegarder le topic
        print(f"Topic {topic_idx}: {', '.join(top_words)}") # affichage lisible du topic
    return topics

topics = show_topics(lda, feature_names, n_top_words=12) #afficher les 12 mots les plus importants pour chaque topics


In [ ]:
## topic dominant par documents (plus ex url)

topic_dist = lda.transform(X) # Calculer la distribution des topics pour chaque document --> une matrice (documents x topics)
df_articles["dominant_topic"] = topic_dist.argmax(axis=1) # Pour chaque article, on sélectionne le topic le plus dominant (celui avec la probabilité la plus élevée)

for t in range(n_topics):
    print(f"\n=== Topic {t} ===")
    print("Exemples d'URLs:")
    print(df_articles[df_articles["dominant_topic"] == t]["url"].head(5).to_list())
    # --> # Sélectionner quelques articles dont le topic dominant est t
    # head(5) permet d'afficher seulement quelques exemples

In [ ]:
## Répartition des topics

# Compter le nombre d'articles associés à chaque topic dominant
# value_counts() → nombre de documents par topic
# sort_index() → afficher les topics dans l'ordre (0, 1, 2, ...)
topic_counts = df_articles["dominant_topic"].value_counts().sort_index()
print(topic_counts)
print("Pourcentages:", (topic_counts / topic_counts.sum()).round(3)) # Calcul de la proportion (en %) de chaque topic dans le corpus


In [ ]:
# ============================================================
# Semantic Analysis – BERTopic (single blog: MindBodyGreen)
# Each document = one blog article
# Goal: discover latent themes and compare with LDA results
# ============================================================

'''from bertopic import BERTopic
from sentence_transformers import SentenceTransformer''' #--> déja ds le code d eury mais utilisé ici

BLOG_FOR_BERT = "mindbodygreen"  # <-- only this blog will be analyzed with BERTopic

# 1) Keep only documents from ONE blog
df_bertopic = df_final[df_final["blog_name"] == BLOG_FOR_BERT].copy()

# 2) Keep only real articles (remove category/tag/short pages)
df_bertopic["word_count"] = df_bertopic["cleaned"].apply(lambda x: len(str(x).split()))
df_bertopic = df_bertopic[df_bertopic["word_count"] >= 400].copy()

# List of documents for BERTopic
docs = df_bertopic["cleaned"].astype(str).tolist()

print(f"BERTopic applied on blog: {BLOG_FOR_BERT}")
print("Number of documents used for BERTopic:", len(docs))

# 3) Load sentence embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# 4) Initialize BERTopic
topic_model = BERTopic(
    language="english",
    min_topic_size=20,
    nr_topics="auto",
    calculate_probabilities=True,
    verbose=True
)

# 5) Fit the model and assign a topic to each document
topics, probs = topic_model.fit_transform(docs, embedding_model=embedding_model)

# 6) Attach topic labels to the dataframe
df_bertopic["bertopic_topic"] = topics

# 7) Display main topics and representative keywords
topic_info = topic_model.get_topic_info()
print("\nTop discovered topics:")
display(topic_info.head(10))

# 8) Show keywords and example URLs for each topic
for t in topic_info["Topic"].tolist():
    if t == -1:
        continue
    print(f"\n=== Topic {t} ===")
    print("Top keywords:", [w for w, _ in topic_model.get_topic(t)[:10]])
    print("Example URLs:")
    print(df_bertopic[df_bertopic["bertopic_topic"] == t]["url"].head(3).to_list())

# 9) Summary of topic distribution --> tables easy to compare with LDA
bertopic_summary = df_bertopic["bertopic_topic"].value_counts().reset_index()
bertopic_summary.columns = ["topic", "n_docs"]
display(bertopic_summary.head(10))

